# Instructions

Ce notebook ne doit contenir que votre script servant à utiliser vos modèles **entrainés** en les testant sur le dataset *fake-test.csv*. Nous devons pouvoir l'exécuter en cliquant sur *Exécution -> Tout exécuter*.

Nous utiliserons ce script pour évaluer votre modèle sur nos propres données d'évaluation par la suite.


# Installation des librairies

In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Importation des librairies

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
import torch
from sklearn.metrics import accuracy_score, classification_report
import os

# Désactivation de wandb

In [3]:
os.environ["WANDB_DISABLED"] = "true"

# Unzip du modèle et on le place dans my_model ici c'est camembert-base.zip mais il faut changer par le nom du fichier si on veut tester un autre fichier

ici j'ai mis bert-base-french-europeana-cased.zip qui est le meilleur modèle que j'ai entraîner

In [4]:
# camembert-base.zip
# bert-base-french-europeana-cased.zip
# roberta-fake-news-classification.zip
# xlm-roberta-base.zip


# Créer le répertoire cible
!mkdir -p my_model
!rm -rf my_model/*

# Décompresser le fichier ZIP
!unzip bert-base-french-europeana-cased.zip -d my_model

Archive:  bert-base-french-europeana-cased.zip
  inflating: my_model/vocab.txt      
  inflating: my_model/tokenizer.json  
  inflating: my_model/tokenizer_config.json  
  inflating: my_model/model.safetensors  
  inflating: my_model/training_args.bin  
  inflating: my_model/special_tokens_map.json  
  inflating: my_model/config.json    


# Chargement du modèle et du tokenizer

In [5]:
model_path = './my_model'

# Charger le modèle et tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Passage en mode évaluation
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Chargement du dataset de test


In [6]:
# Charger le dataset de test
test_file_path = 'fake_test.csv'
dataset_test = load_dataset("csv", data_files=test_file_path, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

# Tokenisation des données

In [7]:
# Tokenisation des données
def preprocess_function(examples):
    return tokenizer(examples['data'], truncation=True, padding=True, max_length=512)

# Appliquer la tokenisation au dataset
tokenized_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/486 [00:00<?, ? examples/s]

# Prediction et affichage de l'accuracy

In [8]:
from transformers import Trainer
from sklearn.metrics import accuracy_score

# Créer un Trainer avec le modèle et le dataset de test
trainer = Trainer(model=model, tokenizer=tokenizer, eval_dataset=tokenized_test)

# Faire des prédictions
predictions = trainer.predict(tokenized_test)
pred_labels = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

# Résultats
print(f"Accuracy: {accuracy_score(true_labels, pred_labels)}")
print(classification_report(true_labels, pred_labels))

<ipython-input-8-4123a5b2a6ac>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer, eval_dataset=tokenized_test)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Accuracy: 0.9711934156378601
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       272
           1       0.99      0.94      0.97       214

    accuracy                           0.97       486
   macro avg       0.97      0.97      0.97       486
weighted avg       0.97      0.97      0.97       486

